# Evaluation  for MsPacman

In [1]:
%matplotlib inline

import os
import sys
sys.path.append(os.path.expanduser("~/libs"))

import cv2
import numpy as np
import tensorflow as tf
import tensortools as tt

from model.frame_prediction import LSTMConv2DPredictionModel

### Hyperparams

In [2]:
# Data
INPUT_SEQ_LENGTH = 8
OUTPUT_SEQ_LENGTH = 8

# Evaluation
EVAL_BATCH_SIZE = 50

In [3]:
# validation while training
OUT_DIR_NAME = "out-eval"
NUM_SAMPLES = 10
GIF_FPS = 5

#### Directory Paths:

In [4]:
ROOT_DIR = "/work/sauterme/"
DATA_DIR = ROOT_DIR + "data"

In [5]:
TRAIN_DIR = ROOT_DIR + "train/pac/ss/2l3i5hp/c326464k533s121bn/wd1e-05/mse/QQ"

assert os.path.exists(TRAIN_DIR)

### Data

In [6]:
dataset_valid = tt.datasets.ms_pacman.MsPacmanValidDataset(DATA_DIR,
                                                           input_seq_length=INPUT_SEQ_LENGTH,
                                                           target_seq_length=OUTPUT_SEQ_LENGTH,
                                                           crop_size=[32, 32])
dataset_test = tt.datasets.ms_pacman.MsPacmanTestDataset(DATA_DIR,
                                                         input_seq_length=INPUT_SEQ_LENGTH,
                                                         target_seq_length=OUTPUT_SEQ_LENGTH,
                                                         crop_size=[32, 32],
                                                         skip_less_movement=False)

File Ms_Pacman.zip has already been extracted.
File Ms_Pacman.zip has already been extracted.


### Runtime

In [7]:
GPU_ID = 7

In [8]:
runtime = tt.core.DefaultRuntime(train_dir=TRAIN_DIR, gpu_devices=[GPU_ID])

In [9]:
runtime.list_checkpoints()

['/work/sauterme/train/pac/ss/2l3i5hp/c326464k533s121bn/wd1e-05/mse/QQ/model.ckpt-100000',
 '/work/sauterme/train/pac/ss/2l3i5hp/c326464k533s121bn/wd1e-05/mse/QQ/model.ckpt-62000',
 '/work/sauterme/train/pac/ss/2l3i5hp/c326464k533s121bn/wd1e-05/mse/QQ/model.ckpt-64000',
 '/work/sauterme/train/pac/ss/2l3i5hp/c326464k533s121bn/wd1e-05/mse/QQ/model.ckpt-66000',
 '/work/sauterme/train/pac/ss/2l3i5hp/c326464k533s121bn/wd1e-05/mse/QQ/model.ckpt-68000',
 '/work/sauterme/train/pac/ss/2l3i5hp/c326464k533s121bn/wd1e-05/mse/QQ/model.ckpt-70000',
 '/work/sauterme/train/pac/ss/2l3i5hp/c326464k533s121bn/wd1e-05/mse/QQ/model.ckpt-72000',
 '/work/sauterme/train/pac/ss/2l3i5hp/c326464k533s121bn/wd1e-05/mse/QQ/model.ckpt-74000',
 '/work/sauterme/train/pac/ss/2l3i5hp/c326464k533s121bn/wd1e-05/mse/QQ/model.ckpt-76000',
 '/work/sauterme/train/pac/ss/2l3i5hp/c326464k533s121bn/wd1e-05/mse/QQ/model.ckpt-78000',
 '/work/sauterme/train/pac/ss/2l3i5hp/c326464k533s121bn/wd1e-05/mse/QQ/model.ckpt-80000',
 '/work/s

In [10]:
CHECKPOINT = tt.core.LATEST_CHECKPOINT

In [11]:
runtime.register_datasets(None, dataset_valid, dataset_test)

In [12]:
runtime.register_model(LSTMConv2DPredictionModel(main_loss=tt.loss.bce))

Consider to restore the **EMA variables** as well when building the model. These might generate worse results in models using batch-normalization, since the shaddow variables might get restored properly...

In [13]:
runtime.build(restore_checkpoint=CHECKPOINT, restore_model_params=True,
              restore_ema_variables=False, eval_mode=True, verbose=True)

Restoring model parameters...

>>> Model:
alpha_ssim_loss   ->  0.0
output_activation  ->  TANH
lstm_cell_clip    ->  None
bias_init         ->  0.0
alpha_main_loss   ->  1.0
alpha_gdl_loss    ->  1.0
lstm_ksize_hidden  ->  (5, 5)
lstm_bn_hidden_hidden  ->  False
lstm_ksize_input  ->  (3, 3)
ksizes            ->  [(5, 5), (3, 3), (3, 3)]
lstm_use_peepholes  ->  True
lstm_bn_input_hidden  ->  False
strides           ->  [(1, 1), (2, 2), (1, 1)]
filters           ->  [32, 64, 64]
lstm_bn_peepholes  ->  False
bn_feature_enc    ->  True
main_loss         ->  MSE
scheduled_sampling_decay_rate  ->  1000.0
bn_feature_dec    ->  True
weight_decay      ->  1e-05
lstm_layers       ->  2

Found 144 update ops.
Selected checkpoint file: /work/sauterme/train/pac/ss/2l3i5hp/c326464k533s121bn/wd1e-05/mse/QQ/model.ckpt-100000
Restoring variables...

--------------------------------------------------------------------------------
conv-encoder/conv1/W:0                                                |  

## Evaluation

In [ ]:
runtime.validate(EVAL_BATCH_SIZE)

In [14]:
runtime.test(EVAL_BATCH_SIZE)

@100000: Starting test (batch-size: 50, dataset-size: 13056):
100%▕██████████▏- 754s - loss: 0.0066 - psnr: 50.1164 - ssim9: 0.9828 - f1-mae: 0.0025 - f1-ssim5: 0.9893 - sharpdiff: 29.9851 - bce: 0.3785 - ssim7: 0.9828 - f2-mae: 0.0031 - ssim: 0.9829 - f2-psnr: 50.5941 - f2-mse: 9.0715e-04 - ssim5: 0.9833 - msssim: 0.9844 - f1-mse: 6.2366e-04 - mae: 0.0040 - rsse: 3.3069 - f1-sharpdiff: 30.0961 - f1-psnr: 50.5197 - mse: 0.0016 - f2-sharpdiff: 30.1816 - f2-ssim5: 0.9860   


## Visualizations
On which dataset we want to test on on the next section.

In [ ]:
dataset_eval = dataset_test

### Random prediction
Either as **binary** (like in training) or **float** (as in raw dataset)...

In [ ]:
def write_animation(dir_path, inputs, targets, predictions, fps):
    concat_tgt = np.concatenate((inputs, targets))
    concat_pred = np.concatenate((inputs, predictions))

    tt.utils.video.write_multi_gif(os.path.join(dir_path, "anim-{:02d}.gif".format(i)),
                                   [concat_tgt, concat_pred],
                                   fps=fps, pad_value=1.0)

    tt.utils.video.write_multi_image_sequence(os.path.join(dir_path, "timeline-{:02d}.png".format(i)),
                                              [concat_tgt, concat_pred],
                                              pad_value=1.0)

def show(inputs, targets, predictions, rows=2):
    tt.visualization.display_batch(inputs, ncols=5, nrows=rows, title="Inputs")
    tt.visualization.display_batch(targets, ncols=5, nrows=rows, title="Targets")
    tt.visualization.display_batch(predictions, ncols=5, nrows=rows, title="Predictions")

In [ ]:
dir_path = os.path.join(runtime.train_dir, OUT_DIR_NAME, "random")

inputs, targets = dataset_eval.get_batch(NUM_SAMPLES)

predictions = runtime.predict(inputs)

show(inputs[0], targets[0], predictions[0])
for i in range(inputs.shape[0]):
    write_animation(dir_path, inputs[i], targets[i], predictions[i], GIF_FPS)

### Specific Predictions TBD !!!
We are using the inputs used in _Gitgub: Adversarial Video Generation_ cropped out of the website. These consist of two sequences...

In [ ]:
SOURCE_PATH = "assets/other/pacman/"

In [ ]:
def read_sequence(dir_path, seq_id):
    image_list = []
    for i in range(INPUT_SEQ_LENGTH+OUTPUT_SEQ_LENGTH):
        image_path = os.path.join(dir_path, str(seq_id), "{:02d}.png".format(i))
        image = tt.utils.image.read(image_path, color_flags = cv2.IMREAD_GRAYSCALE)
        image_list.append(image)
    seq = np.array(image_list)
    seq = seq / 255.0
    seq = np.expand_dims(seq, 0)
    return seq[:,:INPUT_SEQ_LENGTH] , seq[:,INPUT_SEQ_LENGTH:] 

In [ ]:
dir_path = os.path.join(runtime.train_dir, OUT_DIR_NAME, "spec")

for i in range(6):
    inputs, targets = read_sequence(SOURCE_PATH, i)
    predictions = runtime.predict(inputs)
    
    show(inputs[0], targets[0], predictions[0])
    write_animation(dir_path, inputs[0], targets[0], predictions[0], GIF_FPS)

### Fulll Image
Is the FCN model worth it? :) 

In [ ]:
dataset_valid = tt.datasets.ms_pacman.MsPacmanValidDataset(DATA_DIR,
                                                           input_seq_length=INPUT_SEQ_LENGTH,
                                                           target_seq_length=OUTPUT_SEQ_LENGTH,
                                                           skip_less_movement=False,
                                                           random_flip=False)
dataset_test = tt.datasets.ms_pacman.MsPacmanTestDataset(DATA_DIR,
                                                         input_seq_length=INPUT_SEQ_LENGTH,
                                                         target_seq_length=OUTPUT_SEQ_LENGTH,
                                                         skip_less_movement=False,
                                                         random_flip=False)

In [ ]:
dataset_eval = dataset_test

In [ ]:
runtime.register_datasets(None, dataset_valid, dataset_test)
runtime.build(restore_checkpoint=CHECKPOINT, restore_model_params=True,
              restore_ema_variables=False, eval_mode=True)

In [ ]:
dir_path = os.path.join(runtime.train_dir, OUT_DIR_NAME, "full")

inputs, targets = dataset_eval.get_batch(NUM_SAMPLES)

predictions = runtime.predict(inputs)

show(inputs[0], targets[0], predictions[0])
for i in range(inputs.shape[0]):
    write_animation(dir_path, inputs[i], targets[i], predictions[i], GIF_FPS)

### Full Image - Longer Time Range

In [ ]:
# Data
INPUT_SEQ_LENGTH = 8
OUTPUT_SEQ_LENGTH = 24

dataset_valid = tt.datasets.ms_pacman.MsPacmanValidDataset(DATA_DIR,
                                                           input_seq_length=INPUT_SEQ_LENGTH,
                                                           target_seq_length=OUTPUT_SEQ_LENGTH,
                                                           skip_less_movement=False,
                                                           random_flip=False)
dataset_test = tt.datasets.ms_pacman.MsPacmanTestDataset(DATA_DIR,
                                                         input_seq_length=INPUT_SEQ_LENGTH,
                                                         target_seq_length=OUTPUT_SEQ_LENGTH,
                                                         skip_less_movement=False,
                                                         random_flip=False)

In [ ]:
dataset_eval = dataset_test

In [ ]:
runtime.register_datasets(None, dataset_valid, dataset_test)
runtime.build(restore_checkpoint=CHECKPOINT, restore_model_params=True,
              restore_ema_variables=False, eval_mode=True)

In [ ]:
dir_path = os.path.join(runtime.train_dir, OUT_DIR_NAME, "full-long")

inputs, targets = dataset_eval.get_batch(NUM_SAMPLES)

predictions = runtime.predict(inputs)

show(inputs[0], targets[0], predictions[0])
for i in range(inputs.shape[0]):
    write_animation(dir_path, inputs[i], targets[i], predictions[i], GIF_FPS)